In [1]:
import pandas as pd
from tqdm import tqdm  # 用于显示进度条

# 读取获奖记录文件
prize_records = pd.read_csv("Prize-winning paper record.csv", encoding='latin1')

# 读取三个领域的出版物记录
chemistry = pd.read_csv("Chemistry publication record.csv", encoding='latin1')
medicine = pd.read_csv("Medicine publication record.csv", encoding='latin1')
physics = pd.read_csv("Physics publication record.csv", encoding='latin1')

def find_affiliation(prize_df, source_dfs):
    """
    在多个领域的出版物记录中查找获奖论文的机构信息
    
    参数:
    - prize_df: 获奖记录DataFrame (prize_records)
    - source_dfs: 各领域出版物记录的字典，格式为 {领域名称: DataFrame}
    
    返回:
    - 合并了机构信息的获奖记录DataFrame
    """
    result = prize_df.copy()
    result['Affiliation'] = None
    
    for field, source_df in source_dfs.items():
        field_prizes = result[result['Field'] == field].copy()
        if field_prizes.empty:
            continue
        
        # 预处理标题：转为小写、去除常见停用词和标点
        source_df['Title_processed'] = source_df['Title'].astype(str).str.lower()
        source_df['Title_processed'] = source_df['Title_processed'].str.replace('[^\w\s]', '')  # 移除标点
        source_df['Title_processed'] = source_df['Title_processed'].str.replace('\s+', ' ')  # 合并连续空格
        
        # 使用进度条显示匹配进度
        for idx, prize_row in tqdm(field_prizes.iterrows(), desc=f"匹配{field}领域", total=len(field_prizes)):
            if pd.notna(result.at[idx, 'Affiliation']):
                continue
                
            # 精确匹配：先尝试ID+标题完全一致
            if 'Laureate ID' in prize_row and pd.notna(prize_row['Laureate ID']):
                id_matches = source_df[source_df['Laureate ID'] == prize_row['Laureate ID']]
                
                if not id_matches.empty:
                    # 检查标题相似度（简单版本：词重叠率）
                    if 'Title' in prize_row and pd.notna(prize_row['Title']):
                        title_processed = str(prize_row['Title']).lower().replace('[^\w\s]', '').replace('\s+', ' ')
                        title_words = set(title_processed.split())
                        
                        for _, candidate in id_matches.iterrows():
                            candidate_words = set(candidate['Title_processed'].split())
                            overlap = len(title_words.intersection(candidate_words)) / max(len(title_words), len(candidate_words))
                            
                            if overlap >= 0.8:  # 词重叠率超过80%
                                result.at[idx, 'Affiliation'] = candidate['Affiliation']
                                break
                    
                    # 如果标题相似度检查未找到匹配，但ID匹配，仍使用第一条记录
                    if pd.isna(result.at[idx, 'Affiliation']):
                        result.at[idx, 'Affiliation'] = id_matches.iloc[0]['Affiliation']
            
            # 仅标题匹配（无ID或ID匹配失败时）
            if pd.isna(result.at[idx, 'Affiliation']) and 'Title' in prize_row and pd.notna(prize_row['Title']):
                title_processed = str(prize_row['Title']).lower().replace('[^\w\s]', '').replace('\s+', ' ')
                title_words = set(title_processed.split())
                
                # 寻找标题词重叠率最高的记录
                best_match = None
                highest_overlap = 0.5  # 最低阈值
                
                for _, candidate in source_df.iterrows():
                    candidate_words = set(candidate['Title_processed'].split())
                    overlap = len(title_words.intersection(candidate_words)) / max(len(title_words), len(candidate_words))
                    
                    if overlap > highest_overlap:
                        highest_overlap = overlap
                        best_match = candidate
                
                if best_match is not None:
                    result.at[idx, 'Affiliation'] = best_match['Affiliation']
    
    # 处理缺失值
    missing_count = result['Affiliation'].isna().sum()
    print(f"共有{missing_count}条记录未找到机构信息")
    result['Affiliation'] = result['Affiliation'].fillna('Unknown')
    
    return result

# 验证数据列名
required_prize_cols = ['Field', 'Laureate ID', 'Title']
required_source_cols = ['Laureate ID', 'Title', 'Affiliation']

# 检查获奖记录列
missing_prize_cols = [col for col in required_prize_cols if col not in prize_records.columns]
if missing_prize_cols:
    raise ValueError(f"获奖记录文件缺少必要的列: {missing_prize_cols}")

# 检查各领域数据列
source_dfs = {
    'Physics': physics,
    'Chemistry': chemistry,
    'Medicine': medicine
}

for field, df in source_dfs.items():
    missing_cols = [col for col in required_source_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"{field}领域文件缺少必要的列: {missing_cols}")

print("数据列验证通过！")

# 执行匹配
all_prizes_with_affiliation = find_affiliation(prize_records, source_dfs)

# 清洗机构名称
all_prizes_with_affiliation['Affiliation'] = (
    all_prizes_with_affiliation['Affiliation']
    .astype(str)
    .str.lower()
    .str.strip()
    .replace(['nan', ''], 'Unknown')
)

# 保存结果
output_path = "获奖论文_机构信息.csv"
all_prizes_with_affiliation.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"匹配完成，结果已保存至 '{output_path}'")

# 打印匹配成功率
success_rate = 1 - (all_prizes_with_affiliation['Affiliation'] == 'Unknown').mean()
print(f"机构匹配成功率: {success_rate:.2%}")

/Users/powingsze/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


数据列验证通过！


匹配Medicine领域: 100%|███████████████████████| 332/332 [01:12<00:00,  4.59it/s]

共有295条记录未找到机构信息
匹配完成，结果已保存至 '获奖论文_机构信息.csv'
机构匹配成功率: 100.00%
